In [1]:
import multiprocessing as mp
import numpy as np

Sliding Volume Filter (SVF)

$SVF_O = \frac{1}{M}\sum_{a=0}^{2\pi} \sum_{b=0}^{\pi} \max_{R_{min} < r < R_{max}}(\frac{1}{d+1}\sum_{r-d/2}^{r+d/2}VCI_Q)$

Voxel Convergence Index

$VCI_Q = \cos\phi(qx_\rho, qy_\rho, qz_\rho)
    = \cos\phi(\rho\sin{b}\cos{a}, \rho\sin{b}\sin{a}, \rho\cos{b})$

In [2]:
def sliding_filter(rad, d, L, Rmax, Rmin, n, T):
    pass

Integral Numerica Paralelizada - Regra dos Trapezios

In [3]:
def trapezoidal_rule_segment(f, a, b, n):
    """
    Calcula a integral de f(x) em um segmento [a, b] usando a regra dos trapézios com n subintervalos.
    
    :param f: Função a ser integrada
    :param a: Limite inferior do segmento
    :param b: Limite superior do segmento
    :param n: Número de subintervalos no segmento
    :return: Aproximação da integral de f de a até b no segmento
    """
    h = (b - a) / n
    integral = 0.5 * (f(a) + f(b))
    
    for i in range(1, n):
        integral += f(a + i * h)
    
    integral *= h
    return integral


def parallel_trapezoidal_rule(f, a, b, n, num_processes):
    """
    Aproxima a integral de f(x) de a até b usando a regra dos trapézios com n subintervalos em paralelo.
    
    :param f: Função a ser integrada
    :param a: Limite inferior da integração
    :param b: Limite superior da integração
    :param n: Número total de subintervalos
    :param num_processes: Número de processos paralelos
    :return: Aproximação da integral de f de a até b
    """
    pool = mp.Pool(processes=num_processes)
    segment_width = (b - a) / num_processes
    segment_subintervals = n // num_processes
    
    results = []
    for i in range(num_processes):
        segment_a = a + i * segment_width
        segment_b = segment_a + segment_width
        results.append(pool.apply_async(trapezoidal_rule_segment, (f, segment_a, segment_b, segment_subintervals)))
    
    pool.close()
    pool.join()
    
    integral = sum(result.get() for result in results)
    return integral


In [4]:
# Exemplo de uso
def f(x):
    return x**2 - 4*x +8

a = 1  # Limite inferior
b = 5  # Limite superior
n = 1000  # Número de subintervalos
num_processes = 5  # Número de processos paralelos

resultado = parallel_trapezoidal_rule(f, a, b, n, num_processes)
print(f"Aproximação da integral: {resultado}")

Aproximação da integral: 25.333343999999997


In [5]:
def f(x, y):
    return 16 - x**2 - 2 * y**2

def midpoint_rule_segment(f, xi, xf, yi, yf, nx, ny):
    hx = (xf - xi) / nx
    hy = (yf - yi) / ny
    int_aprox = 0
    
    for i in range(nx):
        for j in range(ny):
            deltax = hx
            deltay = hy
            deltaA = deltax * deltay
            xm = xi + i * deltax + deltax / 2
            ym = yi + j * deltay + deltay / 2
            int_aprox += f(xm, ym) * deltaA
    
    return int_aprox

def parallel_midpoint_rule(f, xi, xf, yi, yf, nx, ny, num_processes):
    pool = mp.Pool(processes=num_processes)
    segment_width_x = (xf - xi) / num_processes
    segment_width_y = (yf - yi) / num_processes
    segment_subintervals_x = nx // num_processes
    segment_subintervals_y = ny // num_processes
    
    results = []
    for i in range(num_processes):
        segment_xi = xi + i * segment_width_x
        segment_xf = segment_xi + segment_width_x
        for j in range(num_processes):
            segment_yi = yi + j * segment_width_y
            segment_yf = segment_yi + segment_width_y
            results.append(pool.apply_async(midpoint_rule_segment, (f, segment_xi, segment_xf, segment_yi, segment_yf, segment_subintervals_x, segment_subintervals_y)))
    
    pool.close()
    pool.join()
    
    integral = sum(result.get() for result in results)
    return integral

In [6]:
# # Exemplo de uso
# xi = 0
# xf = 2
# yi = 0
# yf = 2
# nx = 100
# ny = 100
# num_processes = 4

# resultado = parallel_midpoint_rule(f, xi, xf, yi, yf, nx, ny, num_processes)
# sol = 48
# erro = np.abs(sol - resultado)
# print(f"Aproximação da integral: {resultado}")
# print(f"Erro: {erro}")

In [7]:
def f(x, y, z):
    return 12 * x * (y**2) * (z**3)

def midpoint_rule_segment(f, xi, xf, yi, yf, zi, zf, nx, ny, nz):
    hx = (xf - xi) / nx
    hy = (yf - yi) / ny
    hz = (zf - zi) / nz
    int_aprox = 0
    
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                deltax = hx
                deltay = hy
                deltaz = hz
                deltaV = deltax * deltay * deltaz
                xm = xi + i * deltax + deltax / 2
                ym = yi + j * deltay + deltay / 2
                zm = zi + k * deltaz + deltaz / 2
                int_aprox += f(xm, ym, zm) * deltaV
    
    return int_aprox

def parallel_midpoint_rule(f, xi, xf, yi, yf, zi, zf, nx, ny, nz, num_processes):
    pool = mp.Pool(processes=num_processes)
    segment_width_x = (xf - xi) / num_processes
    segment_width_y = (yf - yi) / num_processes
    segment_width_z = (zf - zi) / num_processes
    segment_subintervals_x = nx // num_processes
    segment_subintervals_y = ny // num_processes
    segment_subintervals_z = nz // num_processes
    
    results = []
    for i in range(num_processes):
        segment_xi = xi + i * segment_width_x
        segment_xf = segment_xi + segment_width_x
        for j in range(num_processes):
            segment_yi = yi + j * segment_width_y
            segment_yf = segment_yi + segment_width_y
            for k in range(num_processes):
                segment_zi = zi + k * segment_width_z
                segment_zf = segment_zi + segment_width_z
                results.append(pool.apply_async(midpoint_rule_segment, (f, segment_xi, segment_xf, segment_yi, segment_yf, segment_zi, segment_zf, segment_subintervals_x, segment_subintervals_y, segment_subintervals_z)))
    
    pool.close()
    pool.join()
    
    integral = sum(result.get() for result in results)
    return integral

In [8]:
# Exemplo de uso
xi = -1
xf = 2
yi = 0
yf = 3
zi = 0
zf = 2
nx = 300
ny = 300
nz = 300
num_processes = 3

resultado = parallel_midpoint_rule(f, xi, xf, yi, yf, zi, zf, nx, ny, nz, num_processes)
sol_exata = 648
erro_percentual = (np.abs(sol_exata - resultado) / sol_exata) * 100
print(f"O valor aproximado da integral tripla é: {resultado}")
print(f"Erro percentual: {erro_percentual}")

O valor aproximado da integral tripla é: 647.994600009988
Erro percentual: 0.000833331791983272


In [9]:
def appendSpherical_np(xyz):
    # Cria um novo array com o mesmo número de linhas que xyz, mas com o dobro de colunas
    # As primeiras três colunas serão as coordenadas cartesianas originais
    # As últimas três colunas serão preenchidas com zeros inicialmente
    ptsnew = np.hstack((xyz, np.zeros(xyz.shape)))
    
    # Calcula a soma dos quadrados das coordenadas x e y
    xy = xyz[:, 0]**2 + xyz[:, 1]**2
    
    # Calcula o raio esférico (distância do ponto à origem)
    ptsnew[:, 3] = np.sqrt(xy + xyz[:, 2]**2)
    
    # Calcula o ângulo de elevação (theta) a partir do eixo Z
    # Para elevação definida do eixo Z para baixo
    ptsnew[:, 4] = np.arctan2(np.sqrt(xy), xyz[:, 2])
    # Se preferir elevação definida do plano XY para cima, use a linha abaixo e comente a linha acima
    # ptsnew[:, 4] = np.arctan2(xyz[:, 2], np.sqrt(xy))
    
    # Calcula o ângulo azimutal (phi) no plano XY
    ptsnew[:, 5] = np.arctan2(xyz[:, 1], xyz[:, 0])
    
    return ptsnew

# Exemplo de uso
# Gera um array de 3 milhões de pontos com coordenadas x, y, z aleatórias
pts = np.random.rand(3000000, 3)

# Converte as coordenadas cartesianas para esféricas e armazena o resultado
result = appendSpherical_np(pts)

# Imprime o resultado (opcional, pode ser removido para evitar sobrecarga de impressão)
# print(result)

In [10]:
import numpy as np
import multiprocessing as mp

def f1(x, y, z):
    return 12 * x * (y**2) * (z**3)

def midpoint_rule_segment1(f, xi, xf, yi, yf, zi, zf, nx, ny, nz):
    hx = (xf - xi) / nx
    hy = (yf - yi) / ny
    hz = (zf - zi) / nz
    
    x = np.linspace(xi + hx/2, xf - hx/2, nx)
    y = np.linspace(yi + hy/2, yf - hy/2, ny)
    z = np.linspace(zi + hz/2, zf - hz/2, nz)
    
    X, Y, Z = np.meshgrid(x, y, z, indexing='ij')
    
    deltaV = hx * hy * hz
    integral = np.sum(f(X, Y, Z) * deltaV)
    
    return integral

def parallel_midpoint_rule1(f, xi, xf, yi, yf, zi, zf, nx, ny, nz, num_processes):
    pool = mp.Pool(processes=num_processes)
    
    segment_width_x = (xf - xi) / num_processes
    segment_width_y = (yf - yi) / num_processes
    segment_width_z = (zf - zi) / num_processes
    
    segment_subintervals_x = nx // num_processes
    segment_subintervals_y = ny // num_processes
    segment_subintervals_z = nz // num_processes
    
    results = []
    for i in range(num_processes):
        segment_xi = xi + i * segment_width_x
        segment_xf = segment_xi + segment_width_x
        for j in range(num_processes):
            segment_yi = yi + j * segment_width_y
            segment_yf = segment_yi + segment_width_y
            for k in range(num_processes):
                segment_zi = zi + k * segment_width_z
                segment_zf = segment_zi + segment_width_z
                results.append(pool.apply_async(midpoint_rule_segment1, (f, segment_xi, segment_xf, segment_yi, segment_yf, segment_zi, segment_zf, segment_subintervals_x, segment_subintervals_y, segment_subintervals_z)))
    
    pool.close()
    pool.join()
    
    integral = sum(result.get() for result in results)
    return integral

In [11]:
# # Exemplo de uso
# xi = -1
# xf = 2
# yi = 0
# yf = 3
# zi = 0
# zf = 2
# nx = 300
# ny = 300
# nz = 300
# num_processes = 3

# resultado = parallel_midpoint_rule1(f1, xi, xf, yi, yf, zi, zf, nx, ny, nz, num_processes)
# sol_exata = 648
# erro_percentual = (np.abs(sol_exata - resultado) / sol_exata) * 100
# print(f"O valor aproximado da integral tripla (com numpy) é: {resultado}")
# print(f"Erro percentual: {erro_percentual}")

Nov 12 - Implementação Integral Tripla Paralelizada e Conversão de Coordenadas Cartesianas para Esféricas

In [25]:
import numpy as np
import multiprocessing as mp

def f_spherical(r, theta, phi):
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)
    return 12 * x * (y**2) * (z**2)

def midpoint_rule_segment_spherical(f, r_min, r_max, theta_min, theta_max, phi_min, phi_max, nr, ntheta, nphi):
    hr = (r_max - r_min) / nr
    htheta = (theta_max - theta_min) / ntheta
    hphi = (phi_max - phi_min) / nphi
    
    r = np.linspace(r_min + hr/2, r_max - hr/2, nr)
    theta = np.linspace(theta_min + htheta/2, theta_max - htheta/2, ntheta)
    phi = np.linspace(phi_min + hphi/2, phi_max - hphi/2, nphi)
    
    R, Theta, Phi = np.meshgrid(r, theta, phi, indexing='ij')
    
    deltaV = (R**2) * np.sin(Theta) * hr * htheta * hphi
    integral = np.sum(f(R, Theta, Phi) * deltaV)
    
    return integral

def parallel_midpoint_rule_spherical(f, r_min, r_max, theta_min, theta_max, phi_min, phi_max, nr, ntheta, nphi, num_processes):
    pool = mp.Pool(processes=num_processes)
    
    segment_width_r = (r_max - r_min) / num_processes
    segment_width_theta = (theta_max - theta_min) / num_processes
    segment_width_phi = (phi_max - phi_min) / num_processes
    
    segment_subintervals_r = nr // num_processes
    segment_subintervals_theta = ntheta // num_processes
    segment_subintervals_phi = nphi // num_processes
    
    results = []
    for i in range(num_processes):
        segment_r_min = r_min + i * segment_width_r
        segment_r_max = segment_r_min + segment_width_r
        for j in range(num_processes):
            segment_theta_min = theta_min + j * segment_width_theta
            segment_theta_max = segment_theta_min + segment_width_theta
            for k in range(num_processes):
                segment_phi_min = phi_min + k * segment_width_phi
                segment_phi_max = segment_phi_min + segment_width_phi
                results.append(pool.apply_async(midpoint_rule_segment_spherical, (f, segment_r_min, segment_r_max, segment_theta_min, segment_theta_max, segment_phi_min, segment_phi_max, segment_subintervals_r, segment_subintervals_theta, segment_subintervals_phi)))
    
    pool.close()
    pool.join()
    
    integral = sum(result.get() for result in results)
    return integral

In [29]:
# Exemplo de uso
r_min = 0
r_max = np.sqrt(xf**2 + yf**2 + zf**2)
phi_min = 0
phi_max = np.pi
theta_min = 0
theta_max = 2 * np.pi
nr = 100
ntheta = 100
nphi = 100
num_processes = 4

resultado = parallel_midpoint_rule_spherical(f_spherical, r_min, r_max, theta_min, theta_max, phi_min, phi_max, nr, ntheta, nphi, num_processes)
print(f"O valor aproximado da integral tripla em coordenadas esféricas é: {resultado}")

O valor aproximado da integral tripla em coordenadas esféricas é: -6.821210263296962e-13
